In [11]:
import os
import torch
from tqdm import tqdm
import pandas as pd

In [34]:

DATA_ROOT = '../../dataset'
TEST_DATASET_PATH = os.path.join(DATA_ROOT, 'test')

INFERENCE_SIZE = 1024
AUGMENT = True

IMAGES = []
for image_name in os.listdir(TEST_DATASET_PATH):
    if not image_name.startswith('._'):
        IMAGES.append(os.path.join(TEST_DATASET_PATH, image_name))

IMAGES.sort()


model = torch.hub.load(
    repo_or_dir = './',
    model = 'custom',
    source='local'
    )

model.conf = 0.001  # NMS confidence threshold
model.iou = 0.6  # NMS IoU threshold
# model.agnostic = False  # NMS class-agnostic
# model.multi_label = False  # NMS multiple labels per box
# model.classes = None  # (optional list) filter by class, i.e. = [0, 15, 16] for COCO persons, cats and dogs
# model.max_det = 1000  # maximum number of detections per image
# model.amp = False  # Automatic Mixed Precision (AMP) inference
IMAGES[0]


                 from  n    params  module                                  arguments                     
  0                -1  1       928  models.common.Conv                      [3, 32, 3, 1]                 
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1      8320  models.common.Conv                      [128, 64, 1, 1]               
  5                -2  1      8320  models.common.Conv                      [128, 64, 1, 1]               
  6                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  7                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  8                -1  1     36992  

Adding autoShape... 


'../../dataset/test/0000.jpg'

In [35]:
prediction_strings = []
file_names = []

for i in tqdm(range(len(IMAGES))):
    prediction_string = ''
    
    output = model(IMAGES[i], size=INFERENCE_SIZE, augment=AUGMENT)
    for bbox in output.pandas().xyxy[0].values:
        xmin, ymin, xmax, ymax, conf, cls, _ = bbox
        prediction_string += f'{cls} {conf} {xmin} {ymin} {xmax} {ymax} '
    
    path_split = IMAGES[i].split('/')
    image_path = f'{path_split[-2]}/{path_split[-1]}'

    prediction_strings.append(prediction_string)
    file_names.append(image_path)

submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.head()

100%|██████████| 4871/4871 [09:11<00:00,  8.84it/s]


,PredictionString,image_id
0,7 0.9889755249023438 603.912109375 515.5413818...,test/0000.jpg
1,5 0.9841681718826294 346.16851806640625 251.02...,test/0001.jpg
2,1 0.9622180461883545 295.87152099609375 316.86...,test/0002.jpg
3,9 0.8840042352676392 156.77545166015625 258.22...,test/0003.jpg
4,1 0.7619494199752808 425.87091064453125 407.75...,test/0004.jpg


In [31]:
prediction_strings[1]

'5 0.9850972890853882 345.0753173828125 250.94598388671875 754.8087158203125 695.1448974609375 5 0.9409456253051758 134.0 0.0 492.95556640625 270.0162353515625 6 0.9274826645851135 762.6625366210938 665.074462890625 896.5745239257812 985.6636962890625 9 0.34005868434906006 329.2403869628906 868.0704345703125 472.3697204589844 1023.89208984375 6 0.2692415118217468 658.9832153320312 662.9573974609375 897.9619750976562 982.4576416015625 0 0.051418714225292206 331.47576904296875 867.1309814453125 474.41925048828125 1024.0 7 0.048245370388031006 527.6222534179688 540.5714111328125 550.4863891601562 573.2568359375 0 0.027834508568048477 232.65182495117188 867.1600341796875 473.3615417480469 1024.0 0 0.02480056695640087 1003.3102416992188 713.466064453125 1023.7877807617188 791.995849609375 6 0.019290344789624214 727.535400390625 543.2506713867188 749.4588623046875 569.2897338867188 5 0.01598541811108589 131.05291748046875 0.0349578857421875 283.9277038574219 271.27691650390625 2 0.0142492158

In [36]:
PROJECT_NAME = 'bc_lv2-object_detection'
EXPERIMENT_NAME = 'yolov7_newdata_epoch100_base'
submission.to_csv(os.path.join(f'./{PROJECT_NAME}/{EXPERIMENT_NAME}', 'newCV_base.csv'), index=None)

In [26]:
df = pd.read_csv(os.path.join(f'./{PROJECT_NAME}/{EXPERIMENT_NAME}','originalCV_multiscale.csv'))
df.loc[1]['PredictionString']

'5 0.9607086181640625 345.58282470703125 245.6734161376953 755.9745483398438 699.2506103515625 5 0.8665065765380859 131.24366760253906 0.0 491.6453857421875 274.20172119140625 6 0.7519625425338745 768.1815185546875 660.0931396484375 900.1025390625 986.38720703125 3 0.6226639747619629 344.17327880859375 252.01229858398438 753.1755981445312 695.697021484375 3 0.08237292617559433 147.04832458496094 0.075347900390625 499.26611328125 269.6588439941406 0 0.07113297283649445 650.7095947265625 663.0281982421875 898.718505859375 979.2391357421875 3 0.06663660705089569 631.1354370117188 664.3897705078125 900.5064086914062 1023.500732421875 5 0.050333328545093536 640.8726806640625 663.894775390625 900.3287353515625 1024.0 0 0.029044948518276215 143.9773712158203 0.6124114990234375 500.49432373046875 275.34405517578125 7 0.021789897233247757 0.0 303.4618225097656 302.2508544921875 548.7103271484375 0 0.0192472655326128 738.7014770507812 662.1614990234375 909.8613891601562 1024.0 5 0.01792714186012